# Image Classification Pipeline Summary

## 1. Datasets and Models Used
   - **Datasets**:
     - 4 datasets in total: three individual datasets and one combined dataset.
   - **Models**:
     - VGG16.
     - ResNet50.
     - EfficientNetB0.
     - InceptionV3.

## 2. Experiment Setup
   - **Hyperparameter Ranges**:
     - **Batch Size**: [16, 32, 64, 128].
     - **Learning Rate**: [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005].
     - **Epochs**: [15, 25, 35, 50].
   - Selected optimal hyperparameters for each model based on validation performance.

## 3. Training and Validation Process
   - **Training Loop**:
     - Optimized models over multiple epochs.
     - Logged training and validation accuracy and loss per epoch.
     - Tracked total training time for each model.
   - **Validation**:
     - Monitored model performance with validation data to track generalization and prevent overfitting.

## 4. Evaluation Metrics and Visualizations
   - **Test Set Evaluation**:
     - Assessed model performance using the macro-averaged F1 score.
     - Generated precision, recall, and F1 scores for each class.
   - **Visualizations**:
     - Confusion Matrix for class-wise prediction analysis.
     - Classification Report Heatmap with precision, recall, and F1 scores.
     - ROC Curves for multi-class AUC (Area Under the Curve) evaluation.

## 5. Key Outputs for Each Dataset-Model Combination
   - **Training and Validation Curves**:
     - Generated and saved plots for training and validation accuracy and loss for each model-dataset pairing.
   - **Model State Saving**:
     - Saved trained model states for potential future use.
   - **Detailed Metrics Visualization**:
     - Produced visualizations including classification reports, confusion matrices, and ROC curves for comprehensive performance analysis.


<h1 align="center">Virtual Environment</h1>

In [1]:
# # Create the virtual environment named 'dmp'
!python3 -m venv /scratch/movi/dmp
# Install ipykernel inside the 'dmp' environment
!/scratch/movi/dmp/bin/pip install ipykernel
# Add 'dmp' as a kernel for Jupyter Notebook
!/scratch/movi/dmp/bin/python -m ipykernel install --user --name=dmp --display-name "Python (dmp)"
# # Upgrade pip in the 'dmp' environment
# !/scratch/movi/dmp/bin/python3 -m pip install --upgrade pip
# # Install necessary packages (NumPy, PyTorch, etc.) inside 'dmp'
# !/scratch/movi/dmp/bin/pip install numpy torch torchvision torchaudio pandas matplotlib scikit-learn
# !pip install numpy==1.21.4 scikit-learn==1.0.2

Installed kernelspec dmp in /home/movi/.local/share/jupyter/kernels/dmp


In [2]:
# Prints the installed versions of Python, NumPy, and PyTorch libraries
import sys
import numpy as np
import torch
print(f"Python Version: {sys.version}")
print(f"NumPy Version: {np.__version__}")
print(f"PyTorch Version: {torch.__version__}")

# Function to check GPU availability and display memory statistics using PyTorch's CUDA interface
def check_gpu_status():
    # Check if GPU is available
    if torch.cuda.is_available():
        print(f"CUDA is available. PyTorch is using GPU.\n")
        # Get the number of available GPUs
        num_gpus = torch.cuda.device_count()
        print(f"Number of GPUs available: {num_gpus}")
        # Loop through each GPU and display its details
        for gpu_id in range(num_gpus):
            gpu_name = torch.cuda.get_device_name(gpu_id)
            gpu_memory_allocated = torch.cuda.memory_allocated(gpu_id) / (1024 ** 3)  # In GB
            gpu_memory_cached = torch.cuda.memory_reserved(gpu_id) / (1024 ** 3)      # In GB
            gpu_memory_total = torch.cuda.get_device_properties(gpu_id).total_memory / (1024 ** 3)  # In GB
            print(f"\nGPU {gpu_id}: {gpu_name}")
            print(f"  Total Memory: {gpu_memory_total:.2f} GB")
            print(f"  Memory Allocated: {gpu_memory_allocated:.2f} GB")
            print(f"  Memory Reserved (Cached): {gpu_memory_cached:.2f} GB")
    else:
        print("CUDA is not available. PyTorch is using the CPU.")

# Run the GPU status check
check_gpu_status()

Python Version: 3.9.9 (main, Mar 25 2022, 16:08:31) 
[GCC 10.3.0]
NumPy Version: 1.21.4
PyTorch Version: 1.12.1+cu113
CUDA is available. PyTorch is using GPU.

Number of GPUs available: 1

GPU 0: NVIDIA A100-SXM4-80GB MIG 3g.40gb
  Total Memory: 39.25 GB
  Memory Allocated: 0.00 GB
  Memory Reserved (Cached): 0.00 GB


---

---

<h1 align="center">Dataset 01</h1>

<h2 align="center">VGG16 Model training</h2>

---

In [3]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using VGG16

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset1_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset1_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset1_aug/test'

# Hyperparameters
batch_sizes = [128]
learning_rates = [0.00005]
epoch_counts = [25]
NUM_CLASSES = 9
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.vgg16(weights=None)
            model.classifier[6] = nn.Linear(model.classifier[6].in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Dataset 01 VGG16 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_01_vgg16_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Dataset 01 VGG16 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_01_vgg16_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Dataset 01 VGG16 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_01_vgg16_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Dataset 01 VGG16 Confusion Matrix")
                plt.savefig('dataset_01_vgg16_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Dataset 01 VGG16 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_01_vgg16_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_01_vgg16_model_trained.pth')


2024-11-12 07:47:38,118 - Training dataset size: 10021
2024-11-12 07:47:38,118 - Validation dataset size: 1199
2024-11-12 07:47:38,119 - Test dataset size: 1342
2024-11-12 07:47:38,120 - Training with Batch Size: 128, Learning Rate: 5e-05, Epochs: 25
2024-11-12 07:48:35,074 - Epoch 1/25 - Training: Loss = 1.9407, Accuracy = 0.2608
2024-11-12 07:48:37,918 - Epoch 1/25 - Validation: Loss = 1.6878, Accuracy = 0.3878
2024-11-12 07:48:37,919 - Time for epoch 1: 57.29s
2024-11-12 07:49:26,708 - Epoch 2/25 - Training: Loss = 1.5408, Accuracy = 0.4465
2024-11-12 07:49:29,262 - Epoch 2/25 - Validation: Loss = 1.4193, Accuracy = 0.5263
2024-11-12 07:49:29,263 - Time for epoch 2: 51.34s
2024-11-12 07:50:21,012 - Epoch 3/25 - Training: Loss = 1.2478, Accuracy = 0.5667
2024-11-12 07:50:25,240 - Epoch 3/25 - Validation: Loss = 1.1846, Accuracy = 0.5796
2024-11-12 07:50:25,241 - Time for epoch 3: 55.98s
2024-11-12 07:51:14,560 - Epoch 4/25 - Training: Loss = 0.9671, Accuracy = 0.6613
2024-11-12 07:51

---

<h2 align="center">ResNet50 Model training</h2>

---

In [4]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using ResNet50

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset1_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset1_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset1_aug/test'

# Hyperparameters
batch_sizes = [64]
learning_rates = [0.00005]
epoch_counts = [25]
NUM_CLASSES = 9
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.resnet50(weights=None)
            model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Dataset 01 ResNet50 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_01_resnet50_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Dataset 01 ResNet50 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_01_resnet50_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Dataset 01 ResNet50 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_01_resnet50_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Dataset 01 ResNet50 Confusion Matrix")
                plt.savefig('dataset_01_resnet50_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Dataset 01 ResNet50 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_01_resnet50_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_01_resnet50_model_trained.pth')


2024-11-12 08:11:12,961 - Training dataset size: 10021
2024-11-12 08:11:12,962 - Validation dataset size: 1199
2024-11-12 08:11:12,962 - Test dataset size: 1342
2024-11-12 08:11:12,963 - Training with Batch Size: 64, Learning Rate: 5e-05, Epochs: 25
2024-11-12 08:11:48,688 - Epoch 1/25 - Training: Loss = 1.8871, Accuracy = 0.2992
2024-11-12 08:11:50,827 - Epoch 1/25 - Validation: Loss = 1.6812, Accuracy = 0.3686
2024-11-12 08:11:50,828 - Time for epoch 1: 37.65s
2024-11-12 08:12:29,718 - Epoch 2/25 - Training: Loss = 1.5856, Accuracy = 0.4060
2024-11-12 08:12:31,811 - Epoch 2/25 - Validation: Loss = 1.6096, Accuracy = 0.3778
2024-11-12 08:12:31,812 - Time for epoch 2: 40.98s
2024-11-12 08:13:05,787 - Epoch 3/25 - Training: Loss = 1.4617, Accuracy = 0.4604
2024-11-12 08:13:07,830 - Epoch 3/25 - Validation: Loss = 1.4555, Accuracy = 0.4662
2024-11-12 08:13:07,831 - Time for epoch 3: 36.02s
2024-11-12 08:13:41,944 - Epoch 4/25 - Training: Loss = 1.3269, Accuracy = 0.5111
2024-11-12 08:13:

---

<h2 align="center">EfficientNet_B0 Model training</h2>

---

In [5]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using EfficientNet-B0

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset1_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset1_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset1_aug/test'

# Hyperparameters
batch_sizes = [64]
learning_rates = [0.00005]
epoch_counts = [25]
NUM_CLASSES = 9
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.efficientnet_b0(weights=None)
            model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Dataset 01 EfficientNet_B0 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_01_EfficientNet_B0_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Dataset 01 EfficientNet_B0 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_01_EfficientNet_B0_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Dataset 01 EfficientNet_B0 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_01_EfficientNet_B0_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Dataset 01 EfficientNet_B0 Confusion Matrix")
                plt.savefig('dataset_01_EfficientNet_B0_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Dataset 01 EfficientNet_B0 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_01_EfficientNet_B0_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_01_efficientnet_b0_model_trained.pth')


2024-11-12 08:28:12,062 - Training dataset size: 10021
2024-11-12 08:28:12,063 - Validation dataset size: 1199
2024-11-12 08:28:12,063 - Test dataset size: 1342
2024-11-12 08:28:12,065 - Training with Batch Size: 64, Learning Rate: 5e-05, Epochs: 25
2024-11-12 08:28:43,650 - Epoch 1/25 - Training: Loss = 2.1359, Accuracy = 0.1819
2024-11-12 08:28:45,358 - Epoch 1/25 - Validation: Loss = 2.0013, Accuracy = 0.2294
2024-11-12 08:28:45,359 - Time for epoch 1: 33.22s
2024-11-12 08:29:15,362 - Epoch 2/25 - Training: Loss = 1.8916, Accuracy = 0.2976
2024-11-12 08:29:17,058 - Epoch 2/25 - Validation: Loss = 1.8397, Accuracy = 0.2952
2024-11-12 08:29:17,059 - Time for epoch 2: 31.70s
2024-11-12 08:29:48,771 - Epoch 3/25 - Training: Loss = 1.7419, Accuracy = 0.3584
2024-11-12 08:29:50,465 - Epoch 3/25 - Validation: Loss = 1.6756, Accuracy = 0.3461
2024-11-12 08:29:50,466 - Time for epoch 3: 33.41s
2024-11-12 08:30:20,538 - Epoch 4/25 - Training: Loss = 1.5934, Accuracy = 0.4182
2024-11-12 08:30:

---

<h2 align="center">Inception_V3 Model training</h2>

---

In [6]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using InceptionV3

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset1_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset1_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset1_aug/test'

# Hyperparameters
batch_sizes = [128]
learning_rates = [0.0005]
epoch_counts = [25]
NUM_CLASSES = 9
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.inception_v3(weights=None, aux_logits=False, init_weights=True)
            model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Dataset 01 Inception_V3 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_01_Inception_V3_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Dataset 01 Inception_V3 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_01_Inception_V3_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Dataset 01 Inception_V3 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_01_Inception_V3_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Dataset 01 Inception_V3 Confusion Matrix")
                plt.savefig('dataset_01_Inception_V3_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Dataset 01 Inception_V3 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_01_Inception_V3_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_01_inception_v3_model_trained.pth')


2024-11-12 08:43:38,009 - Training dataset size: 10021
2024-11-12 08:43:38,010 - Validation dataset size: 1199
2024-11-12 08:43:38,010 - Test dataset size: 1342
2024-11-12 08:43:38,012 - Training with Batch Size: 128, Learning Rate: 0.0005, Epochs: 25
2024-11-12 08:44:38,479 - Epoch 1/25 - Training: Loss = 1.6780, Accuracy = 0.3784
2024-11-12 08:44:41,941 - Epoch 1/25 - Validation: Loss = 1.6042, Accuracy = 0.3903
2024-11-12 08:44:41,942 - Time for epoch 1: 60.18s
2024-11-12 08:45:38,617 - Epoch 2/25 - Training: Loss = 1.3025, Accuracy = 0.5223
2024-11-12 08:45:42,128 - Epoch 2/25 - Validation: Loss = 1.1845, Accuracy = 0.5888
2024-11-12 08:45:42,130 - Time for epoch 2: 60.19s
2024-11-12 08:46:38,387 - Epoch 3/25 - Training: Loss = 0.9261, Accuracy = 0.6677
2024-11-12 08:46:41,982 - Epoch 3/25 - Validation: Loss = 0.9335, Accuracy = 0.6664
2024-11-12 08:46:41,983 - Time for epoch 3: 59.85s
2024-11-12 08:47:38,740 - Epoch 4/25 - Training: Loss = 0.6136, Accuracy = 0.7817
2024-11-12 08:4

---

***

<h1 align="center">Dataset 02</h1>

<h2 align="center">VGG16 Model training</h2>

---

In [7]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using VGG16

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset2_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset2_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset2_aug/test'

# Hyperparameters
batch_sizes = [64]
learning_rates = [0.0001]
epoch_counts = [25]
NUM_CLASSES = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.vgg16(weights=None)
            model.classifier[6] = nn.Linear(model.classifier[6].in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Dataset 02 VGG16 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_02_vgg16_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Dataset 02 VGG16 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_02_vgg16_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Dataset 02 VGG16 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_02_vgg16_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Dataset 02 VGG16 Confusion Matrix")
                plt.savefig('dataset_02_vgg16_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Dataset 02 VGG16 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_02_vgg16_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_02_vgg16_model_trained.pth')


2024-11-12 09:14:09,954 - Training dataset size: 8679
2024-11-12 09:14:09,955 - Validation dataset size: 1034
2024-11-12 09:14:09,955 - Test dataset size: 1199
2024-11-12 09:14:09,956 - Training with Batch Size: 64, Learning Rate: 0.0001, Epochs: 25
2024-11-12 09:15:05,256 - Epoch 1/25 - Training: Loss = 1.9505, Accuracy = 0.2923
2024-11-12 09:15:07,712 - Epoch 1/25 - Validation: Loss = 1.5355, Accuracy = 0.4758
2024-11-12 09:15:07,713 - Time for epoch 1: 56.62s
2024-11-12 09:15:53,253 - Epoch 2/25 - Training: Loss = 1.3840, Accuracy = 0.5311
2024-11-12 09:15:55,484 - Epoch 2/25 - Validation: Loss = 1.2367, Accuracy = 0.5977
2024-11-12 09:15:55,485 - Time for epoch 2: 47.77s
2024-11-12 09:16:50,213 - Epoch 3/25 - Training: Loss = 0.9277, Accuracy = 0.6872
2024-11-12 09:16:55,239 - Epoch 3/25 - Validation: Loss = 0.8010, Accuracy = 0.7360
2024-11-12 09:16:55,240 - Time for epoch 3: 59.75s
2024-11-12 09:17:39,792 - Epoch 4/25 - Training: Loss = 0.6788, Accuracy = 0.7681
2024-11-12 09:17:

---

<h2 align="center">ResNet50 Model training</h2>

---

In [8]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using ResNet50

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset2_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset2_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset2_aug/test'

# Hyperparameters
batch_sizes = [64]
learning_rates = [0.0001]
epoch_counts = [25]
NUM_CLASSES = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.resnet50(weights=None)
            model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Dataset 02 ResNet50 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_02_resnet50_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Dataset 02 ResNet50 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_02_resnet50_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Dataset 02 ResNet50 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_02_resnet50_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Dataset 02 ResNet50 Confusion Matrix")
                plt.savefig('dataset_02_resnet50_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Dataset 02 ResNet50 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_02_resnet50_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_02_resnet50_model_trained.pth')


2024-11-12 09:35:10,713 - Training dataset size: 8679
2024-11-12 09:35:10,714 - Validation dataset size: 1034
2024-11-12 09:35:10,714 - Test dataset size: 1199
2024-11-12 09:35:10,715 - Training with Batch Size: 64, Learning Rate: 0.0001, Epochs: 25
2024-11-12 09:35:54,253 - Epoch 1/25 - Training: Loss = 1.8732, Accuracy = 0.3113
2024-11-12 09:35:55,997 - Epoch 1/25 - Validation: Loss = 1.8838, Accuracy = 0.3462
2024-11-12 09:35:55,998 - Time for epoch 1: 45.06s
2024-11-12 09:36:58,113 - Epoch 2/25 - Training: Loss = 1.5382, Accuracy = 0.4464
2024-11-12 09:37:01,958 - Epoch 2/25 - Validation: Loss = 1.5951, Accuracy = 0.4091
2024-11-12 09:37:01,959 - Time for epoch 2: 65.96s
2024-11-12 09:37:53,464 - Epoch 3/25 - Training: Loss = 1.2945, Accuracy = 0.5389
2024-11-12 09:38:00,456 - Epoch 3/25 - Validation: Loss = 1.1864, Accuracy = 0.5667
2024-11-12 09:38:00,457 - Time for epoch 3: 58.50s
2024-11-12 09:38:47,193 - Epoch 4/25 - Training: Loss = 1.0711, Accuracy = 0.6200
2024-11-12 09:38:

---

<h2 align="center">EfficientNet_B0 Model training</h2>

---

In [9]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using EfficientNet-B0

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset2_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset2_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset2_aug/test'

# Hyperparameters
batch_sizes = [64]
learning_rates = [0.0001]
epoch_counts = [25]
NUM_CLASSES = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.efficientnet_b0(weights=None)
            model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Dataset 02 EfficientNet_B0 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_02_EfficientNet_B0_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Dataset 02 EfficientNet_B0 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_02_EfficientNet_B0_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Dataset 02 EfficientNet_B0 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_02_EfficientNet_B0_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Dataset 02 EfficientNet_B0 Confusion Matrix")
                plt.savefig('dataset_02_EfficientNet_B0_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Dataset 02 EfficientNet_B0 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_02_EfficientNet_B0_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_02_efficientnet_b0_model_trained.pth')


2024-11-12 09:50:55,786 - Training dataset size: 8679
2024-11-12 09:50:55,786 - Validation dataset size: 1034
2024-11-12 09:50:55,787 - Test dataset size: 1199
2024-11-12 09:50:55,788 - Training with Batch Size: 64, Learning Rate: 0.0001, Epochs: 25
2024-11-12 09:51:21,986 - Epoch 1/25 - Training: Loss = 2.1090, Accuracy = 0.2306
2024-11-12 09:51:23,380 - Epoch 1/25 - Validation: Loss = 1.8878, Accuracy = 0.3172
2024-11-12 09:51:23,381 - Time for epoch 1: 27.51s
2024-11-12 09:51:49,772 - Epoch 2/25 - Training: Loss = 1.8048, Accuracy = 0.3381
2024-11-12 09:51:51,927 - Epoch 2/25 - Validation: Loss = 1.7354, Accuracy = 0.3897
2024-11-12 09:51:51,928 - Time for epoch 2: 28.55s
2024-11-12 09:52:24,200 - Epoch 3/25 - Training: Loss = 1.5720, Accuracy = 0.4352
2024-11-12 09:52:29,049 - Epoch 3/25 - Validation: Loss = 1.4618, Accuracy = 0.4632
2024-11-12 09:52:29,050 - Time for epoch 3: 37.12s
2024-11-12 09:52:59,936 - Epoch 4/25 - Training: Loss = 1.3375, Accuracy = 0.5211
2024-11-12 09:53:

---

<h2 align="center">Inception_V3 Model training</h2>

---

In [10]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using InceptionV3

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset2_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset2_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset2_aug/test'

# Hyperparameters
batch_sizes = [128]
learning_rates = [0.0005]
epoch_counts = [25]
NUM_CLASSES = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.inception_v3(weights=None, aux_logits=False, init_weights=True)
            model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Dataset 02 Inception_V3 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_02_Inception_V3_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Dataset 02 Inception_V3 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_02_Inception_V3_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Dataset 02 Inception_V3 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_02_Inception_V3_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Dataset 02 Inception_V3 Confusion Matrix")
                plt.savefig('dataset_02_Inception_V3_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Dataset 02 Inception_V3 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_02_Inception_V3_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_02_inception_v3_model_trained.pth')


2024-11-12 10:04:56,255 - Training dataset size: 8679
2024-11-12 10:04:56,256 - Validation dataset size: 1034
2024-11-12 10:04:56,256 - Test dataset size: 1199
2024-11-12 10:04:56,258 - Training with Batch Size: 128, Learning Rate: 0.0005, Epochs: 25
2024-11-12 10:05:48,995 - Epoch 1/25 - Training: Loss = 1.7009, Accuracy = 0.3870
2024-11-12 10:05:52,066 - Epoch 1/25 - Validation: Loss = 1.5402, Accuracy = 0.4342
2024-11-12 10:05:52,067 - Time for epoch 1: 55.48s
2024-11-12 10:06:40,497 - Epoch 2/25 - Training: Loss = 1.2991, Accuracy = 0.5267
2024-11-12 10:06:44,465 - Epoch 2/25 - Validation: Loss = 1.5201, Accuracy = 0.5261
2024-11-12 10:06:44,466 - Time for epoch 2: 52.40s
2024-11-12 10:07:33,421 - Epoch 3/25 - Training: Loss = 0.9934, Accuracy = 0.6434
2024-11-12 10:07:37,134 - Epoch 3/25 - Validation: Loss = 0.9782, Accuracy = 0.6644
2024-11-12 10:07:37,135 - Time for epoch 3: 52.67s
2024-11-12 10:08:32,676 - Epoch 4/25 - Training: Loss = 0.7386, Accuracy = 0.7365
2024-11-12 10:08

---

---

<h1 align="center">Dataset 03</h1>

<h2 align="center">VGG16 Model training</h2>

---

In [11]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using VGG16

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset3_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset3_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset3_aug/test'

# Hyperparameters
batch_sizes = [128]
learning_rates = [0.00001]
epoch_counts = [25]
NUM_CLASSES = 8
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.vgg16(weights=None)
            model.classifier[6] = nn.Linear(model.classifier[6].in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Dataset 03 VGG16 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_03_vgg16_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Dataset 03 VGG16 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_03_vgg16_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Dataset 03 VGG16 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_03_vgg16_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Dataset 03 VGG16 Confusion Matrix")
                plt.savefig('dataset_03_vgg16_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Dataset 03 VGG16 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_03_vgg16_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_03_vgg16_model_trained.pth')


2024-11-12 10:28:51,383 - Training dataset size: 11594
2024-11-12 10:28:51,384 - Validation dataset size: 1408
2024-11-12 10:28:51,384 - Test dataset size: 1529
2024-11-12 10:28:51,386 - Training with Batch Size: 128, Learning Rate: 1e-05, Epochs: 25
2024-11-12 10:30:09,520 - Epoch 1/25 - Training: Loss = 1.9355, Accuracy = 0.2364
2024-11-12 10:30:14,989 - Epoch 1/25 - Validation: Loss = 1.5127, Accuracy = 0.4766
2024-11-12 10:30:14,990 - Time for epoch 1: 82.47s
2024-11-12 10:31:12,808 - Epoch 2/25 - Training: Loss = 1.2974, Accuracy = 0.4799
2024-11-12 10:31:16,087 - Epoch 2/25 - Validation: Loss = 1.0105, Accuracy = 0.6072
2024-11-12 10:31:16,088 - Time for epoch 2: 61.10s
2024-11-12 10:32:18,585 - Epoch 3/25 - Training: Loss = 0.9802, Accuracy = 0.5994
2024-11-12 10:32:21,925 - Epoch 3/25 - Validation: Loss = 0.8099, Accuracy = 0.7045
2024-11-12 10:32:21,926 - Time for epoch 3: 65.84s
2024-11-12 10:33:20,954 - Epoch 4/25 - Training: Loss = 0.8503, Accuracy = 0.6610
2024-11-12 10:33

---

<h2 align="center">ResNet50 Model training</h2>

---

In [12]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using ResNet50

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset3_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset3_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset3_aug/test'

# Hyperparameters
batch_sizes = [64]
learning_rates = [0.00001]
epoch_counts = [25]
NUM_CLASSES = 8
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.resnet50(weights=None)
            model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Dataset 03 ResNet50 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_03_resnet50_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Dataset 03 ResNet50 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_03_resnet50_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Dataset 03 ResNet50 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_03_resnet50_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Dataset 03 ResNet50 Confusion Matrix")
                plt.savefig('dataset_03_resnet50_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Dataset 03 ResNet50 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_03_resnet50_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_03_resnet50_model_trained.pth')


2024-11-12 10:57:12,016 - Training dataset size: 11594
2024-11-12 10:57:12,016 - Validation dataset size: 1408
2024-11-12 10:57:12,017 - Test dataset size: 1529
2024-11-12 10:57:12,018 - Training with Batch Size: 64, Learning Rate: 1e-05, Epochs: 25
2024-11-12 10:57:56,150 - Epoch 1/25 - Training: Loss = 2.0167, Accuracy = 0.2268
2024-11-12 10:57:58,806 - Epoch 1/25 - Validation: Loss = 1.8713, Accuracy = 0.3324
2024-11-12 10:57:58,807 - Time for epoch 1: 46.57s
2024-11-12 10:58:38,367 - Epoch 2/25 - Training: Loss = 1.5591, Accuracy = 0.4195
2024-11-12 10:58:41,155 - Epoch 2/25 - Validation: Loss = 1.1964, Accuracy = 0.5483
2024-11-12 10:58:41,156 - Time for epoch 2: 42.35s
2024-11-12 10:59:23,058 - Epoch 3/25 - Training: Loss = 1.0518, Accuracy = 0.5762
2024-11-12 10:59:25,570 - Epoch 3/25 - Validation: Loss = 0.8667, Accuracy = 0.6428
2024-11-12 10:59:25,571 - Time for epoch 3: 44.41s
2024-11-12 11:00:08,553 - Epoch 4/25 - Training: Loss = 0.8707, Accuracy = 0.6375
2024-11-12 11:00:

---

<h2 align="center">EfficientNet_B0 Model training</h2>

---

In [3]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using EfficientNet-B0

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset3_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset3_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset3_aug/test'

# Hyperparameters
batch_sizes = [64]
learning_rates = [0.00001]
epoch_counts = [25]
NUM_CLASSES = 8
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.efficientnet_b0(weights=None)
            model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Dataset 03 EfficientNet_B0 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_03_EfficientNet_B0_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Dataset 03 EfficientNet_B0 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_03_EfficientNet_B0_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Dataset 03 EfficientNet_B0 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_03_EfficientNet_B0_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Dataset 03 EfficientNet_B0 Confusion Matrix")
                plt.savefig('dataset_03_EfficientNet_B0_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Dataset 03 EfficientNet_B0 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_03_EfficientNet_B0_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_03_efficientnet_b0_model_trained.pth')


2024-11-14 07:38:36,834 - Training dataset size: 11594
2024-11-14 07:38:36,834 - Validation dataset size: 1408
2024-11-14 07:38:36,835 - Test dataset size: 1529
2024-11-14 07:38:36,836 - Training with Batch Size: 64, Learning Rate: 1e-05, Epochs: 25
2024-11-14 07:39:33,492 - Epoch 1/25 - Training: Loss = 2.0624, Accuracy = 0.1768
2024-11-14 07:39:37,866 - Epoch 1/25 - Validation: Loss = 2.0435, Accuracy = 0.1918
2024-11-14 07:39:37,867 - Time for epoch 1: 59.48s
2024-11-14 07:40:14,805 - Epoch 2/25 - Training: Loss = 2.0260, Accuracy = 0.2089
2024-11-14 07:40:17,118 - Epoch 2/25 - Validation: Loss = 1.9970, Accuracy = 0.2230
2024-11-14 07:40:17,119 - Time for epoch 2: 39.25s
2024-11-14 07:40:54,715 - Epoch 3/25 - Training: Loss = 1.9069, Accuracy = 0.2757
2024-11-14 07:40:56,954 - Epoch 3/25 - Validation: Loss = 1.7035, Accuracy = 0.3232
2024-11-14 07:40:56,955 - Time for epoch 3: 39.84s
2024-11-14 07:41:33,877 - Epoch 4/25 - Training: Loss = 1.6862, Accuracy = 0.3314
2024-11-14 07:41:

---

<h2 align="center">Inception_V3 Model training</h2>

---

In [4]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using InceptionV3

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset3_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset3_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset3_aug/test'

# Hyperparameters
batch_sizes = [64]
learning_rates = [0.00001]
epoch_counts = [25]
NUM_CLASSES = 8
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.inception_v3(weights=None, aux_logits=False, init_weights=True)
            model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Dataset 03 Inception_V3 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_03_Inception_V3_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Dataset 03 Inception_V3 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_03_Inception_V3_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Dataset 03 Inception_V3 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_03_Inception_V3_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Dataset 03 Inception_V3 Confusion Matrix")
                plt.savefig('dataset_03_Inception_V3_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Dataset 03 Inception_V3 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_03_Inception_V3_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_03_inception_v3_model_trained.pth')


2024-11-14 07:55:40,900 - Training dataset size: 11594
2024-11-14 07:55:40,901 - Validation dataset size: 1408
2024-11-14 07:55:40,901 - Test dataset size: 1529
2024-11-14 07:55:40,903 - Training with Batch Size: 64, Learning Rate: 1e-05, Epochs: 25
2024-11-14 07:56:48,662 - Epoch 1/25 - Training: Loss = 1.9835, Accuracy = 0.2305
2024-11-14 07:56:52,972 - Epoch 1/25 - Validation: Loss = 1.8199, Accuracy = 0.3814
2024-11-14 07:56:52,973 - Time for epoch 1: 71.72s
2024-11-14 07:58:00,053 - Epoch 2/25 - Training: Loss = 1.6878, Accuracy = 0.4061
2024-11-14 07:58:04,445 - Epoch 2/25 - Validation: Loss = 1.4483, Accuracy = 0.5256
2024-11-14 07:58:04,446 - Time for epoch 2: 71.47s
2024-11-14 08:00:06,121 - Epoch 3/25 - Training: Loss = 1.3439, Accuracy = 0.5154
2024-11-14 08:00:11,816 - Epoch 3/25 - Validation: Loss = 1.1438, Accuracy = 0.6101
2024-11-14 08:00:11,818 - Time for epoch 3: 127.37s
2024-11-14 08:02:08,682 - Epoch 4/25 - Training: Loss = 1.1130, Accuracy = 0.5731
2024-11-14 08:02

---

---

<h1 align="center">Combined Dataset</h1>

<h2 align="center">VGG16 Model training</h2>

---

In [15]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using VGG16

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset4_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset4_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset4_aug/test'

# Hyperparameters
batch_sizes = [64]
learning_rates = [0.00001]
epoch_counts = [25]
NUM_CLASSES = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.vgg16(weights=None)
            model.classifier[6] = nn.Linear(model.classifier[6].in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Combined Dataset VGG16 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_04_vgg16_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Combined Dataset VGG16 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_04_vgg16_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Combined Dataset VGG16 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_04_vgg16_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Combined Dataset VGG16 Confusion Matrix")
                plt.savefig('dataset_04_vgg16_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Combined Dataset VGG16 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_04_vgg16_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_04_vgg16_model_trained.pth')


2024-11-12 12:04:52,334 - Training dataset size: 21802
2024-11-12 12:04:52,335 - Validation dataset size: 2673
2024-11-12 12:04:52,335 - Test dataset size: 2827
2024-11-12 12:04:52,337 - Training with Batch Size: 64, Learning Rate: 1e-05, Epochs: 25
2024-11-12 12:07:19,746 - Epoch 1/25 - Training: Loss = 1.7890, Accuracy = 0.3299
2024-11-12 12:07:29,521 - Epoch 1/25 - Validation: Loss = 1.3709, Accuracy = 0.4860
2024-11-12 12:07:29,523 - Time for epoch 1: 156.05s
2024-11-12 12:09:26,261 - Epoch 2/25 - Training: Loss = 1.3098, Accuracy = 0.5333
2024-11-12 12:09:33,155 - Epoch 2/25 - Validation: Loss = 1.2753, Accuracy = 0.5305
2024-11-12 12:09:33,156 - Time for epoch 2: 123.63s
2024-11-12 12:11:28,858 - Epoch 3/25 - Training: Loss = 1.0973, Accuracy = 0.6188
2024-11-12 12:11:35,860 - Epoch 3/25 - Validation: Loss = 1.1293, Accuracy = 0.5918
2024-11-12 12:11:35,861 - Time for epoch 3: 122.70s
2024-11-12 12:13:31,750 - Epoch 4/25 - Training: Loss = 0.9202, Accuracy = 0.6799
2024-11-12 12:

---

<h2 align="center">ResNet50 Model training</h2>

---

In [16]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using ResNet50

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset4_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset4_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset4_aug/test'

# Hyperparameters
batch_sizes = [128]
learning_rates = [0.00005]
epoch_counts = [25]
NUM_CLASSES = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.resnet50(weights=None)
            model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Combined Dataset ResNet50 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_04_resnet50_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Combined Dataset ResNet50 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_04_resnet50_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Combined Dataset ResNet50 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_04_resnet50_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Combined Dataset ResNet50 Confusion Matrix")
                plt.savefig('dataset_04_resnet50_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Combined Dataset ResNet50 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_04_resnet50_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_04_resnet50_model_trained.pth')


2024-11-12 12:57:28,455 - Training dataset size: 21802
2024-11-12 12:57:28,456 - Validation dataset size: 2673
2024-11-12 12:57:28,456 - Test dataset size: 2827
2024-11-12 12:57:28,458 - Training with Batch Size: 128, Learning Rate: 5e-05, Epochs: 25
2024-11-12 12:58:41,925 - Epoch 1/25 - Training: Loss = 1.7063, Accuracy = 0.3695
2024-11-12 12:58:46,768 - Epoch 1/25 - Validation: Loss = 1.6191, Accuracy = 0.4074
2024-11-12 12:58:46,769 - Time for epoch 1: 78.09s
2024-11-12 13:00:01,946 - Epoch 2/25 - Training: Loss = 1.2438, Accuracy = 0.5499
2024-11-12 13:00:06,804 - Epoch 2/25 - Validation: Loss = 1.1753, Accuracy = 0.5769
2024-11-12 13:00:06,806 - Time for epoch 2: 80.03s
2024-11-12 13:01:20,467 - Epoch 3/25 - Training: Loss = 1.0146, Accuracy = 0.6399
2024-11-12 13:01:25,215 - Epoch 3/25 - Validation: Loss = 0.9116, Accuracy = 0.6831
2024-11-12 13:01:25,217 - Time for epoch 3: 78.41s
2024-11-12 13:02:39,667 - Epoch 4/25 - Training: Loss = 0.8084, Accuracy = 0.7192
2024-11-12 13:02

---

<h2 align="center">EfficientNet_B0 Model training</h2>

---

In [17]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using EfficientNet-B0

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset4_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset4_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset4_aug/test'

# Hyperparameters
batch_sizes = [128]
learning_rates = [0.00005]
epoch_counts = [25]
NUM_CLASSES = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.efficientnet_b0(weights=None)
            model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Combined Dataset EfficientNet_B0 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_04_EfficientNet_B0_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Combined Dataset EfficientNet_B0 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_04_EfficientNet_B0_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Combined Dataset EfficientNet_B0 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_04_EfficientNet_B0_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Combined Dataset EfficientNet_B0 Confusion Matrix")
                plt.savefig('dataset_04_EfficientNet_B0_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Combined Dataset EfficientNet_B0 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_04_EfficientNet_B0_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_04_efficientnet_b0_model_trained.pth')


2024-11-12 13:30:51,127 - Training dataset size: 21802
2024-11-12 13:30:51,128 - Validation dataset size: 2673
2024-11-12 13:30:51,128 - Test dataset size: 2827
2024-11-12 13:30:51,129 - Training with Batch Size: 128, Learning Rate: 5e-05, Epochs: 25
2024-11-12 13:32:11,076 - Epoch 1/25 - Training: Loss = 2.1180, Accuracy = 0.2041
2024-11-12 13:32:17,783 - Epoch 1/25 - Validation: Loss = 1.8279, Accuracy = 0.3008
2024-11-12 13:32:17,785 - Time for epoch 1: 86.58s
2024-11-12 13:33:28,487 - Epoch 2/25 - Training: Loss = 1.7019, Accuracy = 0.3564
2024-11-12 13:33:32,934 - Epoch 2/25 - Validation: Loss = 1.5024, Accuracy = 0.4572
2024-11-12 13:33:32,936 - Time for epoch 2: 75.15s
2024-11-12 13:34:43,602 - Epoch 3/25 - Training: Loss = 1.3890, Accuracy = 0.4968
2024-11-12 13:34:48,684 - Epoch 3/25 - Validation: Loss = 1.1935, Accuracy = 0.5630
2024-11-12 13:34:48,686 - Time for epoch 3: 75.75s
2024-11-12 13:35:58,734 - Epoch 4/25 - Training: Loss = 1.1630, Accuracy = 0.5865
2024-11-12 13:36

---

<h2 align="center">Inception_V3 Model training</h2>

---

In [18]:
# Written by Ovi, 2024-11-11
# Image classification pipeline using InceptionV3

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
import numpy as np
import random
from itertools import cycle
from sklearn.preprocessing import label_binarize

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths
train_dir = '/scratch/movi/dm_project/data/split_80/dataset4_aug/train'
val_dir = '/scratch/movi/dm_project/data/split_80/dataset4_aug/val'
test_dir = '/scratch/movi/dm_project/data/split_80/dataset4_aug/test'

# Hyperparameters
batch_sizes = [64]
learning_rates = [0.0001]
epoch_counts = [25]
NUM_CLASSES = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

logger.info(f"Training dataset size: {len(train_dataset)}")
logger.info(f"Validation dataset size: {len(val_dataset)}")
logger.info(f"Test dataset size: {len(test_dataset)}")

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    train_acc_history, val_acc_history = [], []
    train_loss_history, val_loss_history = [], []
    total_training_time = 0
    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
        train_loss = 0
        train_correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc.cpu())

        logger.info(f"Epoch {epoch + 1}/{epochs} - Training: Loss = {train_loss:.4f}, Accuracy = {train_acc:.4f}")

        model.eval()
        val_loss = 0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)
        val_loss_history.append(val_loss)
        val_acc_history.append(val_acc.cpu())

        epoch_time = time.time() - epoch_start_time
        total_training_time += epoch_time

        logger.info(f"Epoch {epoch + 1}/{epochs} - Validation: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")
        logger.info(f"Time for epoch {epoch + 1}: {epoch_time:.2f}s")

    logger.info(f"Total Training Time: {total_training_time:.2f}s")

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history

for batch_size in batch_sizes:
    for lr in learning_rates:
        for epochs in epoch_counts:
            logger.info(f"Training with Batch Size: {batch_size}, Learning Rate: {lr}, Epochs: {epochs}")

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            model = models.inception_v3(weights=None, aux_logits=False, init_weights=True)
            model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
            model = model.to(DEVICE)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            train_acc_history, val_acc_history, train_loss_history, val_loss_history = train_and_validate(
                model, train_loader, val_loader, criterion, optimizer, epochs
            )

            epochs_range = range(1, epochs + 1)
            plt.figure()
            plt.plot(epochs_range, train_acc_history, label='Training Accuracy')
            plt.plot(epochs_range, val_acc_history, label='Validation Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.title(f'Combined Dataset Inception_V3 Accuracy (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_04_Inception_V3_accuracy_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            plt.figure()
            plt.plot(epochs_range, train_loss_history, label='Training Loss')
            plt.plot(epochs_range, val_loss_history, label='Validation Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title(f'Combined Dataset Inception_V3 Loss (Batch Size {batch_size}, LR {lr}, Epochs {epochs})')
            plt.legend()
            plt.savefig(f'dataset_04_Inception_V3_loss_batch_{batch_size}_lr_{lr}_epochs_{epochs}.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
            plt.close()

            def test_and_evaluate(model, test_loader, class_names):
                model.eval()
                all_labels = []
                all_preds = []

                with torch.no_grad():
                    for inputs, labels in test_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        all_labels.extend(labels.cpu().numpy())
                        all_preds.extend(preds.cpu().numpy())

                macro_f1 = f1_score(all_labels, all_preds, average='macro')
                logger.info(f"Macro-Averaged F1 Score: {macro_f1:.4f}")

                precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=np.unique(all_labels))
                metrics_df = np.array([precision, recall, f1]).T
                plt.figure(figsize=(8, 6))
                sns.heatmap(metrics_df, annot=True, cmap="viridis", xticklabels=["Precision", "Recall", "F1"], yticklabels=class_names)
                plt.title("Combined Dataset Inception_V3 Classification Report")
                plt.xlabel("Metric")
                plt.ylabel("Class")
                plt.savefig('dataset_04_Inception_V3_classification_report.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                cm = confusion_matrix(all_labels, all_preds)
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
                plt.xlabel("Predicted Label")
                plt.ylabel("True Label")
                plt.title("Combined Dataset Inception_V3 Confusion Matrix")
                plt.savefig('dataset_04_Inception_V3_confusion_matrix.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

                all_labels_binarized = label_binarize(all_labels, classes=np.arange(NUM_CLASSES))
                all_preds_binarized = np.eye(NUM_CLASSES)[np.array(all_preds)]

                plt.figure(figsize=(10, 8))
                colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'cyan'])
                for i, color in zip(range(NUM_CLASSES), colors):
                    fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], all_preds_binarized[:, i])
                    roc_auc = auc(fpr, tpr)
                    plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc:.2f})')

                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('Combined Dataset Inception_V3 ROC Curves for All Classes')
                plt.legend(loc="lower right")
                plt.savefig('dataset_04_Inception_V3_ROC_All_Classes.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
                plt.close()

            test_and_evaluate(model, test_loader, class_names=test_dataset.classes)

            torch.save(model.state_dict(), 'dataset_04_inception_v3_model_trained.pth')


2024-11-12 14:02:45,680 - Training dataset size: 21802
2024-11-12 14:02:45,681 - Validation dataset size: 2673
2024-11-12 14:02:45,681 - Test dataset size: 2827
2024-11-12 14:02:45,683 - Training with Batch Size: 64, Learning Rate: 0.0001, Epochs: 25
2024-11-12 14:04:59,007 - Epoch 1/25 - Training: Loss = 1.4714, Accuracy = 0.4627
2024-11-12 14:05:06,757 - Epoch 1/25 - Validation: Loss = 1.1926, Accuracy = 0.5593
2024-11-12 14:05:06,758 - Time for epoch 1: 140.80s
2024-11-12 14:07:12,590 - Epoch 2/25 - Training: Loss = 0.9756, Accuracy = 0.6521
2024-11-12 14:07:20,260 - Epoch 2/25 - Validation: Loss = 0.8669, Accuracy = 0.6891
2024-11-12 14:07:20,262 - Time for epoch 2: 133.50s
2024-11-12 14:09:29,371 - Epoch 3/25 - Training: Loss = 0.7513, Accuracy = 0.7381
2024-11-12 14:09:37,760 - Epoch 3/25 - Validation: Loss = 0.7306, Accuracy = 0.7235
2024-11-12 14:09:37,761 - Time for epoch 3: 137.50s
2024-11-12 14:11:46,736 - Epoch 4/25 - Training: Loss = 0.5979, Accuracy = 0.7914
2024-11-12 14

---

---